In [ ]:
from placekey.api import PlacekeyAPI
from ast import literal_eval
import snowflake.connector
from snowflake.connector import cursor
import usaddress
import re
import pandas as pd
# from helper_functions import *
import config_final

## Dealing with Abbreviations!

In [ ]:
ctx = snowflake.connector.connect(
    user=config_final.user,
    password=config_final.password,
    account=config_final.account
)
cursor = ctx.cursor()

sql_MLS = """ """
df_MLS = pd.read_sql(sql_MLS, con=ctx)
sql_DATATREE = """  """
df_DATATREE = pd.read_sql(sql_DATATREE, con=ctx)


In [ ]:
# Drop NaN
df_MLS.dropna(subset=['MLS_ADDRESS'], inplace=True)
df_DATATREE.dropna(subset=['DATATREE_ADDRESS'], inplace=True)

# df_MLS = clean_special_char(df_MLS,'MLS_ADDRESS')
# df_DATATREE = clean_special_char(df_DATATREE,'DATATREE_ADDRESS')

df1_MLS = df_MLS.copy()
df1_DATATREE = df_DATATREE.copy()

df1_MLS['MLS_ADDRESS_Split'] = df1_MLS.apply(lambda df1_MLS : get_splited_addr(df1_MLS,'MLS_ADDRESS'), axis = 1)
df1_DATATREE['DATATREE_ADDRESS_Split'] = df1_DATATREE.apply(lambda df1_DATATREE : get_splited_addr(df1_DATATREE,'DATATREE_ADDRESS'), axis = 1)



In [ ]:
MLS_ADDRESS_final = get_comb_addr(df1_MLS,'MLS_ADDRESS_Split')
DT_ADDRESS_final = get_comb_addr(df1_DATATREE,'DATATREE_ADDRESS_Split')

# convert address format (direction, abbreviation)
MLS_ADDRESS_final,DT_ADDRESS_final=get_str_replace(MLS_ADDRESS_final,DT_ADDRESS_final)


In [ ]:
from placekey.api import PlacekeyAPI
placekey_api_key = "n1w1KOroOdSz3BVMMlg8IM4PuflcVP4V" # fill this in with your personal API key (do not share publicly)
pk_api = PlacekeyAPI(placekey_api_key)
column_map = {"DATATREE_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")


In [ ]:
DT_ADDRESS_final.columns

In [ ]:
DT_ADDRESS_final.SITUSUNITTYPE.unique()

In [ ]:
DT_ADDRESS_final.OccupancyType.unique()

In [ ]:
UnitType = DT_ADDRESS_final[['DATATREE_ADDRESS','SITUSUNITTYPE','OccupancyType']]

In [ ]:
UnitType[(UnitType['SITUSUNITTYPE']!=UnitType['OccupancyType'])&(UnitType['SITUSUNITTYPE']=='ph')]


## Topic: Do we want to have abbreviation transformation in the original dataset?

Conclusion: we can! But whether it's necessary to do it will depend on what we're gonna do later!

#### If unit signal exist:
For Accessor Table:\
there's no difference in the keys generated by converted/non-converted combined columns
=> we can use combined column to generate keys! it doesn't matter whether we transform those directional words/ street type or not
=> depending on what we're gonna do in the later column-bu-column string mappings! Yay!

For MLS Table:\
there's no difference in the keys generated by converted combined columns vs. original column
=> it's okay to standatdize directional words AND StreetTypes
=> but we keep the order of ['AddressNumber', 'StreetNamePreDirectional','StreetNamePreType' ,'StreetName','StreetNamePostType','StreetNamePostDirectional','OccupancyType', 'OccupancyIdentifier']

#### If unit signal does not exist:
we can substitute with the unit signal which occurs the most => 57/745 successfully generated new keys based on data in NYC

### case1: when unit signal exists

In [ ]:
def get_str_replace(MLS_ADDRESS_final,DT_ADDRESS_final):
    #need to change file path
    df = pd.read_csv('/Users/yiweihou/Downloads/Roc360/suffix_abbreviations_converted.csv')
    StreetNamePostType_dict = dict(zip(df.common_name, df.abbr))
    StreetNameDirectional_dict = {'west': 'w', 'south': 's', 'north': 'n','east': 'e'}

    DT_ADDRESS_final.replace({"StreetNamePreDirectional": StreetNameDirectional_dict},inplace=True)
    DT_ADDRESS_final.replace({"StreetNamePostType": StreetNamePostType_dict},inplace=True)

    MLS_ADDRESS_final.replace({"StreetNamePreDirectional": StreetNameDirectional_dict},inplace=True)
    MLS_ADDRESS_final.replace({"StreetNamePostType": StreetNamePostType_dict},inplace=True)
    return MLS_ADDRESS_final,DT_ADDRESS_final

if there is no abbr tranformation, i.e., we follow whatever the original data looks like

In [ ]:
test_noTraAbbr = DT_ADDRESS_final.copy()
test_noTraAbbr['ColumnA'] = test_noTraAbbr[test_noTraAbbr.columns[1:8]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

# for this step, we only test addresses with unit number!
test_DT_noTranAbbr = test_noTraAbbr[~test_noTraAbbr['SITUSUNITTYPE'].isna()][:1000]
# generate key with ColumnA-after string tranformation
# => no matter if we remove special characters or not, there is no missing info if we use ColumnA ( since we are using the original columns)
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_DT_noTranAbbr,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_noTranAbbr = pd.concat([df_placekey,test_DT_noTranAbbr.reset_index()],axis=1)


if we transform abbr

In [ ]:
# DT_ADDRESS_final[DT_ADDRESS_final['SITUSUNITTYPE']=='ph']
# since unittype and unit number is always not the same, we might need to be careful when joining splited strings together?

 
df = pd.read_csv('suffix_abbreviations_converted.csv')
StreetNamePostType_dict = dict(zip(df.common_name, df.abbr))

test_TranAbbr = DT_ADDRESS_final.copy()

StreetNameDirectional_dict = {'west': 'w', 'south': 's', 'north': 'n','east': 'e'}

test_TranAbbr.replace({"SITUSDIRECTIONLEFT": StreetNameDirectional_dict},inplace=True)
test_TranAbbr.replace({"SITUSDIRECTIONRIGHT": StreetNameDirectional_dict},inplace=True)

test_TranAbbr.replace({"SITUSMODE": StreetNamePostType_dict},inplace=True)


test_TranAbbr['ColumnA'] = test_TranAbbr[test_TranAbbr.columns[1:8]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
test_TranAbbr = test_TranAbbr[~test_TranAbbr['SITUSUNITTYPE'].isna()][:1000]
# generate key with ColumnA-after string tranformation
# => no matter if we remove special characters or not, there is no missing info if we use ColumnA ( since we are using the original columns)
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_TranAbbr,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_Abbr = pd.concat([df_placekey,test_TranAbbr.reset_index()],axis=1)


In [ ]:
concat_noTranAbbr.head(5)

In [ ]:
concat_Abbr.head(5)

In [ ]:
# compare if abbr matters
IfAbbrMatter = pd.merge(concat_noTranAbbr[['query_id','error','placekey','ColumnA','DATATREE_ADDRESS']],concat_Abbr[['query_id','error','placekey','ColumnA']], how = 'left', right_on = ['query_id'], 
                          left_on = ['query_id'])

IfAbbrMatter[(IfAbbrMatter['placekey_x']!=IfAbbrMatter['placekey_y'])&IfAbbrMatter['placekey_x'].notna()]

# no difference

Conclusion: encoding directional words and streetname doesn't impact placekey. If such transformation is good for further address matching, we can definitely incorporate this step in the Assessor Table!

look in mls table


In [ ]:
# still for addresses with unit number 

test_noTraAbbr_mls = MLS_ADDRESS_final.copy()
# keep usaddresses in the same order as how the original string is formatted
test_noTraAbbr_mls['ColumnA'] = test_noTraAbbr_mls[['AddressNumber', 'StreetNamePreDirectional', 'StreetName',
       'StreetNamePostType','StreetNamePostDirectional','OccupancyType', 'OccupancyIdentifier']].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
test_noTraAbbr_mls = test_noTraAbbr_mls[~test_noTraAbbr_mls['OccupancyType'].isna()][:1000]

# generate key with combined ColumnA 
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_noTraAbbr_mls,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_noTranAbbr_mls = pd.concat([df_placekey,test_noTraAbbr_mls.reset_index()],axis=1)


In [ ]:
concat_noTranAbbr_mls[concat_noTranAbbr_mls['MLS_ADDRESS']!=concat_noTranAbbr_mls['ColumnA']]

In [ ]:
# no transfer => meaning that we will use original address column which has been cleaned
test_noTraAbbr_mls = MLS_ADDRESS_final.copy()
test_noTraAbbr_mls = test_noTraAbbr_mls[~test_noTraAbbr_mls['OccupancyType'].isna()][:1000]

# generate key with combined ColumnA 
column_map = {"MLS_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_noTraAbbr_mls,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_noTranAbbr_mls = pd.concat([df_placekey,test_noTraAbbr_mls.reset_index()],axis=1)
# concat_noTranAbbr_mls

In [ ]:
concat_noTranAbbr_mls[concat_noTranAbbr_mls['MLS_ADDRESS']=='196 avenue of the americas apt 1c']
concat_noTranAbbr_mls


In [ ]:
test_TraAbbr_mls = MLS_ADDRESS_final.copy()
test_TraAbbr_mls = test_TraAbbr_mls[~test_TraAbbr_mls['OccupancyType'].isna()][:1000]

test_TraAbbr_mls['ColumnA'] = test_TraAbbr_mls[['AddressNumber', 'StreetNamePreDirectional','StreetNamePreType' ,'StreetName',
       'StreetNamePostType','StreetNamePostDirectional','OccupancyType', 'OccupancyIdentifier']].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

# test_TraAbbr_mls

In [ ]:
# generate key with combined ColumnA 
column_map = {"ColumnA" : "street_address", "POSTALCODE": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_TraAbbr_mls,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_TranAbbr_mls = pd.concat([df_placekey,test_TraAbbr_mls.reset_index()],axis=1)
concat_TranAbbr_mls

In [ ]:
# compare if abbr matters
IfAbbrMatter_mls = pd.merge(concat_noTranAbbr_mls[['query_id','placekey','MLS_ADDRESS']],concat_TranAbbr_mls[['query_id','placekey','ColumnA']], how = 'left', right_on = ['query_id'], 
                          left_on = ['query_id'])
IfAbbrMatter_mls
IfAbbrMatter_mls[(IfAbbrMatter_mls['placekey_x']!=IfAbbrMatter_mls['placekey_y'])& (IfAbbrMatter_mls['placekey_x'].notna())]

# yay seems that there's no difference!




In [ ]:
# concat_noTranAbbr_mls[concat_noTranAbbr_mls['MLS_ADDRESS']=='60 sutton pl s 14gs']

Conclusion: it's okay to use converted address to generate keys!
### If unit signal exist:
1. For Accessor Table:\
there's no difference in the keys generated by converted/non-converted combined columns \
=> we can use combined column to  generate keys! it doesn't matter whether we transform those directional words/ street type or not \
=> depending on what we're gonna do in the later column-bu-column string mappings! Yay!

2. For MLS Table:\
there's no difference in the keys generated by converted combined columns vs. original column\
=> it's okay to standatdize directional words AND StreetTypes\
=> but we keep the order of ['AddressNumber', 'StreetNamePreDirectional','StreetNamePreType' ,'StreetName','StreetNamePostType','StreetNamePostDirectional','OccupancyType', 'OccupancyIdentifier']



### case2: when there is no signal

In [ ]:
# we take original tabel without abbreviation transformation
test_noTraAbbr[(test_noTraAbbr['SITUSUNITNBR'].notna())].shape # address with unit number
# shared building? 
# TODO: generate sample-> a building with mul units in the same building 




maybe get placekey id for each building (if keys being returned are the same for add. which only diff. in the unit numbers)

In [ ]:
# DT_ADDRESS_final[(DT_ADDRESS_final['SITUSUNITTYPE'].isna())&(DT_ADDRESS_final['SITUSUNITNBR'].notna())]
# concat_Abbr
noUnitNum = test_noTraAbbr[(test_noTraAbbr['SITUSUNITTYPE'].isna())&(test_noTraAbbr['SITUSUNITNBR'].notna())]

noUnitNum

# generate key with ColumnA-after string tranformation
# => no matter if we remove special characters or not, there is no missing info if we use ColumnA ( since we are using the original columns)
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(noUnitNum,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
noUnitNum_key1 = pd.concat([df_placekey,noUnitNum.reset_index()],axis=1)


In [ ]:
noUnitNum

In [ ]:
test_noTraAbbr.shape

In [ ]:
# old_df.loc[old_df.product_url.isin(list(to_zero)), 'availability'] = 0
noUnitNum.loc[noUnitNum['SITUSUNITTYPE'].isna(),'SITUSUNITTYPE'] = 'apt'

In [ ]:
noUnitNum['ColumnA_new'] = noUnitNum[noUnitNum.columns[1:8]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

column_map = {"ColumnA_new" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(noUnitNum,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
noUnitNum_key2 = pd.concat([df_placekey,noUnitNum.reset_index()],axis=1)


In [ ]:
# final result for address which have unit number but do not have unit signals 
# => we add a signal where they are the same as the unit type occuring the most frequently 
add_unitsig = pd.merge(noUnitNum_key1[['query_id','placekey','DATATREE_ADDRESS','ColumnA']],noUnitNum_key2[['query_id','placekey','ColumnA_new']], how = 'left', right_on = ['query_id'], 
                          left_on = ['query_id'])

add_unitsig[(add_unitsig['placekey_x']!=add_unitsig['placekey_y'])].shape

# do we need to manually add a signal?


57/745 -> new york

In [ ]:
add_unitsig[(add_unitsig['placekey_x']!=add_unitsig['placekey_y'])]

In [ ]:
# df_DATATREE
DT_ADDRESS_final
# DT_ADDRESS_final['SITUSUNITTYPE'].value_counts()
# OccupancyType
test_noTraAbbr['SITUSUNITTYPE'].value_counts()

In [ ]:
DT_ADDRESS_final['SITUSUNITTYPE'].value_counts()

for addresses that have unit signal:

In [ ]:
test_DT = test_noTraAbbr[~test_noTraAbbr['SITUSUNITTYPE'].isna()][:1000]
# test_noTraAbbr[(test_noTraAbbr['SITUSUNITNBR'].notna())].shape # address with unit number


In [ ]:
# generate key with ColumnA-after string tranformation
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_DT,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_ColA = pd.concat([df_placekey,test_DT.reset_index()],axis=1)


In [ ]:
# generate key with original data - after string tranformation
column_map = {"DATATREE_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_DT,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_Orig =pd.concat([df_placekey,test_DT.reset_index()],axis=1)


In [ ]:
concat_ColA[['query_id','error','placekey','DATATREE_ADDRESS','ColumnA']]

In [ ]:
concat_Orig[['query_id','error','placekey','DATATREE_ADDRESS','ColumnA']]